<a href="https://colab.research.google.com/github/NakanoDataLab/data_analysis_100knocks/blob/main/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第2章　***小売店のデータでデータ加工を行う10本ノック***(11～20)


 ノック11　データを読み込んでみよう

In [1]:
from google.colab import drive
import os
import pandas as pd

# Google Drive をマウント
drive.mount('/content/drive')

# 作業ディレクトリの設定
DATA_ROOT_DIR = "/content/drive/MyDrive/data_analysis_knock/"
os.chdir(DATA_ROOT_DIR)

# google colab でexsxを扱うためのライブラリをインストール
!pip install openpyxl

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# データの読み込み
uriage_data = pd.read_csv("uriage.csv")

# データの確認
print('売上データの先頭5行')
print(uriage_data.head())

売上データの先頭5行
         purchase_date item_name  item_price customer_name
0  2019-04-13 00:54:07       商品X      2400.0         長谷川 舞
1  2019-01-21 00:36:30       商品I         NaN        三浦 さゆり
2  2019-05-01 14:22:42       商品p      1600.0         山田 千代
3  2019-02-22 11:32:36       商品L      1200.0          高橋 学
4  2019-07-13 21:36:05       商品J      1000.0          林 香織


In [3]:
#データの読み込み(顧客データ)
kokyaku_data = pd.read_excel("kokyaku_data.xlsx", engine="openpyxl")

# データの確認
print("顧客データの先頭5行")
print(kokyaku_data.head())

顧客データの先頭5行
      顧客名      かな  地域             メールアドレス                  登録日
0   遠藤 洋介  サンプルかな  X市  sample@example.com                43134
1   村上 亮介  サンプルかな  X市  sample@example.com  2018-02-28 00:00:00
2  佐々木 千代  サンプルかな  X市  sample@example.com  2018-03-31 00:00:00
3   山田 千代  サンプルかな  X市  sample@example.com  2018-04-30 00:00:00
4   清水 直子  サンプルかな  X市  sample@example.com  2018-05-31 00:00:00


        ノック12 データの揺れを見てみる

In [4]:
#　商品の揺れを確認
print(uriage_data['item_name'].head())
print(uriage_data['item_price'].head())

0    商品X
1    商品I
2    商品p
3    商品L
4    商品J
Name: item_name, dtype: object
0    2400.0
1       NaN
2    1600.0
3    1200.0
4    1000.0
Name: item_price, dtype: float64


        ノック13 揺れがあるまま集計してみよう

In [5]:
# 日付データをdatetime型に変換
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])

# 月ごとに計算できるように "purchase_month" 列を作成
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")

# 商品ごとの購入数を集計（揺れがあるまま）
res = uriage_data.pivot_table(
    index="purchase_month",     # indexは"purchase_month"
    columns="item_name",        # columnsは"item_name"
    aggfunc="size",             # 件数をカウント
    fill_value=0                # 欠損値は0で埋める
)
# 結果を表示
res

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品q,商品r,商品s,商品t,商品u,商品v,商品w,商品x,商品y,商品z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,7,17,9,15,18,17,18,21,17,14,...,3,2,0,2,4,1,2,1,1,1
201902,10,16,11,20,16,19,9,11,15,14,...,0,0,2,1,0,2,0,1,0,3
201903,15,15,7,15,19,19,17,16,16,13,...,1,0,0,5,2,3,3,3,2,1
201904,19,19,15,14,20,18,17,17,14,13,...,0,2,1,2,2,4,2,3,0,0
201905,15,9,12,15,19,20,15,14,14,18,...,1,4,1,0,2,4,1,2,1,3
201906,14,22,11,17,10,16,14,21,16,23,...,0,1,1,0,0,1,2,1,3,1
201907,13,14,13,19,17,20,11,11,14,13,...,3,1,0,2,0,2,2,1,3,2


In [6]:
res = uriage_data.pivot_table(
    index="purchase_month",       # 行方向にする（インデックス） → 月ごとに集計
    columns="item_name",          # 列方向にする（カラム） → 商品ごとに集計
    values="item_price",          # 集計する対象 → 商品価格
    aggfunc="sum",                # 集計方法 → 売上金額を合計
    fill_value=0                  # データがない部分は0で埋める
)
res

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品q,商品r,商品s,商品t,商品u,商品v,商品w,商品x,商品y,商品z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,600.0,3000.0,2700.0,4800.0,9000.0,7800.0,10500.0,13600.0,12600.0,10000.0,...,3400.0,3600.0,0.0,4000.0,6300.0,2200.0,4600.0,2400.0,2500.0,2600.0
201902,800.0,2800.0,2400.0,7200.0,7500.0,9600.0,5600.0,8000.0,12600.0,12000.0,...,0.0,0.0,3800.0,2000.0,0.0,4400.0,0.0,2400.0,0.0,5200.0
201903,1300.0,3000.0,2100.0,5200.0,8500.0,10200.0,10500.0,11200.0,13500.0,11000.0,...,1700.0,0.0,0.0,10000.0,4200.0,4400.0,6900.0,2400.0,5000.0,2600.0
201904,1900.0,3400.0,4200.0,4800.0,8000.0,10200.0,11900.0,12000.0,9900.0,13000.0,...,0.0,3600.0,1900.0,4000.0,4200.0,6600.0,4600.0,7200.0,0.0,0.0
201905,1200.0,1200.0,3300.0,4800.0,9000.0,10800.0,10500.0,10400.0,11700.0,15000.0,...,1700.0,7200.0,1900.0,0.0,4200.0,8800.0,2300.0,4800.0,2500.0,7800.0
201906,1400.0,3600.0,3300.0,6000.0,4000.0,8400.0,8400.0,15200.0,10800.0,21000.0,...,0.0,1800.0,0.0,0.0,0.0,2200.0,4600.0,2400.0,7500.0,2600.0
201907,1200.0,2200.0,3600.0,6800.0,8500.0,9600.0,7000.0,7200.0,10800.0,11000.0,...,5100.0,1800.0,0.0,4000.0,0.0,2200.0,4600.0,2400.0,7500.0,5200.0


ノック14 商品名の揺れを補正しよう

In [7]:
# 商品名にユニークな数字がどのくらいあるのかを調べる
print(len(pd.unique(uriage_data["item_name"])))

52


In [8]:
# カラム内のアルファベットを大文字に揃える
uriage_data["item_name"] = uriage_data["item_name"].str.upper()

# カラム内の全角スペースを半角に揃える
uriage_data["item_name"] = uriage_data["item_name"].str.replace("　", "")

# カラム内のスペースを無くす
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")

# カラム内の順番を昇順に並べ替える
uriage_data.sort_values(by="item_name", ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
1246,2019-01-10 17:04:00,商品A,100.0,石井 淳,201901
911,2019-05-11 04:32:16,商品A,NaN,福田 幹,201905
350,2019-03-21 12:05:43,商品A,NaN,坂本 太郎,201903
1368,2019-06-13 10:29:52,商品A,100.0,高橋 浩,201906
1793,2019-04-21 21:16:23,商品A,100.0,斎藤 くみ子,201904
...,...,...,...,...,...
1310,2019-05-11 16:38:34,商品Z,2600.0,鈴木 裕太,201905
1309,2019-05-06 11:19:02,商品Z,2600.0,青木 太郎,201905
2023,2019-07-16 16:22:54,商品Z,2600.0,青木 裕美子,201907
2892,2019-07-22 10:05:24,商品Z,2600.0,長谷川 七夏,201907


In [9]:
# クリーニング後のデータを確認
print(sorted(pd.unique(uriage_data['item_name'])))
# クリーニング後のデータの件数を確認
print(len(pd.unique(uriage_data['item_name'])))

['商品A', '商品B', '商品C', '商品D', '商品E', '商品F', '商品G', '商品H', '商品I', '商品J', '商品K', '商品L', '商品M', '商品N', '商品O', '商品P', '商品Q', '商品R', '商品S', '商品T', '商品U', '商品V', '商品W', '商品X', '商品Y', '商品Z']
26


        **ノック15** 欠損値の補完をしよう！

In [10]:
#商品単価の欠損値を確認する
uriage_data.isnull().any(axis =0)

,0
purchase_date,False
item_name,False
item_price,True
customer_name,False
purchase_month,False


In [11]:
# NaNの場所をフラグで取得
flag_isnull = uriage_data["item_price"].isnull()

# NaNの商品名をリストで取得
target_list = list(uriage_data.loc[flag_isnull, "item_name"].unique())

for target in target_list:
    # NaN以外の最大価格を取得
    price = uriage_data.loc[
        (~flag_isnull) & (uriage_data["item_name"] == target), "item_price"
    ].max()

    # 取得した最大価格でNaNを補完
    uriage_data.loc[
        (flag_isnull) & (uriage_data["item_name"] == target), "item_price"
    ] = price

In [12]:
#欠損値がないか、もう一度チェック

uriage_data.isnull().any(axis=0)

,0
purchase_date,False
item_name,False
item_price,False
customer_name,False
purchase_month,False


In [13]:
item_name_list = sorted(uriage_data["item_name"].unique())
for item_name in item_name_list:
    price_list = uriage_data.loc[uriage_data["item_name"] == item_name]["item_price"]
    min_val = price_list.min()
    max_val = price_list.max()
    print(item_name, "max :", max_val, "min :", min_val)

商品A max : 100.0 min : 100.0
商品B max : 200.0 min : 200.0
商品C max : 300.0 min : 300.0
商品D max : 400.0 min : 400.0
商品E max : 500.0 min : 500.0
商品F max : 600.0 min : 600.0
商品G max : 700.0 min : 700.0
商品H max : 800.0 min : 800.0
商品I max : 900.0 min : 900.0
商品J max : 1000.0 min : 1000.0
商品K max : 1100.0 min : 1100.0
商品L max : 1200.0 min : 1200.0
商品M max : 1300.0 min : 1300.0
商品N max : 1400.0 min : 1400.0
商品O max : 1500.0 min : 1500.0
商品P max : 1600.0 min : 1600.0
商品Q max : 1700.0 min : 1700.0
商品R max : 1800.0 min : 1800.0
商品S max : 1900.0 min : 1900.0
商品T max : 2000.0 min : 2000.0
商品U max : 2100.0 min : 2100.0
商品V max : 2200.0 min : 2200.0
商品W max : 2300.0 min : 2300.0
商品X max : 2400.0 min : 2400.0
商品Y max : 2500.0 min : 2500.0
商品Z max : 2600.0 min : 2600.0


        **ノック16** 顧客名の揺れを補正しよう

In [14]:
# kokyaku_dataの確認
kokyaku_data['顧客名'].head()

,顧客名
0,遠藤 洋介
1,村上 亮介
2,佐々木 千代
3,山田 千代
4,清水 直子


In [15]:
# uriage_dataの確認
uriage_data['customer_name'].head()

,customer_name
0,長谷川 舞
1,三浦 さゆり
2,山田 千代
3,高橋 学
4,林 香織


In [16]:
#全角、半角の統一
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace("　", "").str.replace(" ", "")
uriage_data['customer_name'] = uriage_data['customer_name'].str.replace("　", "").str.replace(" ", "")

In [17]:
# データの型を指定(str_文字列)
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].astype(str)
uriage_data['customer_name'] = uriage_data['customer_name'].astype(str)

        **ノック17** 日付の揺れを補正しよう

In [18]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,遠藤洋介,サンプルかな,X市,sample@example.com,43134
1,村上亮介,サンプルかな,X市,sample@example.com,2018-02-28 00:00:00
2,佐々木千代,サンプルかな,X市,sample@example.com,2018-03-31 00:00:00
3,山田千代,サンプルかな,X市,sample@example.com,2018-04-30 00:00:00
4,清水直子,サンプルかな,X市,sample@example.com,2018-05-31 00:00:00


In [19]:
# 登録日を文字列に変換し、数値だけで構成されている値をチェック
flag_isserial = kokyaku_data['登録日'].astype('str').str.isdigit()
# 数値だけで構成されている値の件数
flag_isserial.sum()

np.int64(1)

In [20]:
# シリアルを通常の日付に変更
fromSerial = pd.to_timedelta(kokyaku_data.loc[flag_isserial, "登録日"].astype("float"), unit="D") + pd.to_datetime("1900/01/01")
fromSerial

,登録日
0,2018-02-05


In [21]:
# 修正した日付をdatetime型に変更
fromString = pd.to_datetime(kokyaku_data.loc[~flag_isserial, "登録日"])
fromString

,登録日
1,2018-02-28
2,2018-03-31
3,2018-04-30
4,2018-05-31
5,2018-06-30
6,2018-07-31
7,2018-08-31
8,2018-09-30
9,2018-10-31
10,2018-11-30


In [22]:
# 日付データを統一して上書き
kokyaku_data["登録日"] = pd.concat([fromSerial, fromString])
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,遠藤洋介,サンプルかな,X市,sample@example.com,2018-02-05
1,村上亮介,サンプルかな,X市,sample@example.com,2018-02-28
2,佐々木千代,サンプルかな,X市,sample@example.com,2018-03-31
3,山田千代,サンプルかな,X市,sample@example.com,2018-04-30
4,清水直子,サンプルかな,X市,sample@example.com,2018-05-31


In [23]:
# 登録年月のカラムを作成
kokyaku_data["登録年月"] = kokyaku_data["登録日"].dt.strftime("%Y%m")

# 登録年月ごとに顧客名の件数をカウント
rslt = kokyaku_data.groupby("登録年月").count()["顧客名"]

print(rslt)
print(len(kokyaku_data))

登録年月
201802    2
201803    1
201804    1
201805    1
201806    1
201807    1
201808    1
201809    1
201810    1
201811    1
201812    1
201901    1
201902    1
201903    1
201904    1
201905    1
201906    1
201907    1
201908    1
201909    1
201910    1
201911    1
201912    1
202001    1
202002    1
202003    1
202004    1
202005    1
202006    1
202007    1
202008    1
202009    1
202010    1
202011    1
202012    1
202101    1
202102    1
202103    1
202104    1
202105    1
202106    1
202107    1
202108    1
202109    1
202110    1
202111    1
202112    1
202201    1
202202    1
Name: 顧客名, dtype: int64
50


In [24]:
# 登録日のカラムを文字列に変換
flag_isserial = kokyaku_data["登録日"].astype("str").str.isdigit()

# 登録日が数値の列を数える
flag_isserial.sum()

np.int64(0)

ノック18 顧客名をキーに2つのデータを結合(ジョイン)しよう!

In [42]:
# 処理した顧客名とcustomer_nameをジョインしよう!
inner_join_data = pd.merge(uriage_data, kokyaku_data,
                           left_on='customer_name',
                           right_on='顧客名',
                           how='inner')
inner_join_data


,purchase_date,item_name,item_price,customer_name,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-04-13 00:54:07,商品X,2400.0,長谷川舞,201904,長谷川舞,サンプルかな,X市,sample@example.com,2021-10-31,202110
1,2019-05-01 14:22:42,商品P,1600.0,山田千代,201905,山田千代,サンプルかな,X市,sample@example.com,2018-04-30,201804
2,2019-05-02 09:04:01,商品E,500.0,鈴木真綾,201905,鈴木真綾,サンプルかな,B市,sample@example.com,2021-01-31,202101
3,2019-04-10 03:01:42,商品T,2000.0,伊藤康弘,201904,伊藤康弘,サンプルかな,B市,sample@example.com,2019-04-30,201904
4,2019-05-09 07:55:30,商品E,500.0,渡辺陽一,201905,渡辺陽一,サンプルかな,A市,sample@example.com,2022-02-28,202202
...,...,...,...,...,...,...,...,...,...,...,...
801,2019-05-15 08:45:12,商品E,500.0,遠藤洋介,201905,遠藤洋介,サンプルかな,Z市,sample@example.com,2018-02-05,201802
802,2019-04-27 19:28:07,商品J,1000.0,清水涼平,201904,清水涼平,サンプルかな,B市,sample@example.com,2020-10-31,202010
803,2019-01-22 11:39:36,商品K,1100.0,伊藤康弘,201901,伊藤康弘,サンプルかな,B市,sample@example.com,2019-04-30,201904
804,2019-01-17 20:03:45,商品T,2000.0,山本聡太郎,201901,山本聡太郎,サンプルかな,Z市,sample@example.com,2018-08-31,201808


        **ノック19** クレンジングしたデータをダンプしよう！

In [26]:
# 変数dump_dataに処理したデータを代入する
dump_data = inner_join_data[["purchase_date", "purchase_month", "item_name", "item_price", "顧客名", "かな", "地域", "メールアドレス", "登録日"]]
dump_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-04-13 00:54:07,201904,商品X,2400.0,長谷川舞,サンプルかな,X市,sample@example.com,2021-10-31
1,2019-05-01 14:22:42,201905,商品P,1600.0,山田千代,サンプルかな,X市,sample@example.com,2018-04-30
2,2019-05-02 09:04:01,201905,商品E,500.0,鈴木真綾,サンプルかな,X市,sample@example.com,2021-01-31
3,2019-04-10 03:01:42,201904,商品T,2000.0,伊藤康弘,サンプルかな,X市,sample@example.com,2019-04-30
4,2019-05-09 07:55:30,201905,商品E,500.0,渡辺陽一,サンプルかな,X市,sample@example.com,2022-02-28
...,...,...,...,...,...,...,...,...,...
801,2019-05-15 08:45:12,201905,商品E,500.0,遠藤洋介,サンプルかな,X市,sample@example.com,2018-02-05
802,2019-04-27 19:28:07,201904,商品J,1000.0,清水涼平,サンプルかな,X市,sample@example.com,2020-10-31
803,2019-01-22 11:39:36,201901,商品K,1100.0,伊藤康弘,サンプルかな,X市,sample@example.com,2019-04-30
804,2019-01-17 20:03:45,201901,商品T,2000.0,山本聡太郎,サンプルかな,X市,sample@example.com,2018-08-31


In [34]:
# CSVファイルへの書き込み処理(ワークスペース'/content/')
dump_data.to_csv("dump_data_mine.csv", index=False)

        ノック20 データを集計しよう

In [28]:
# CSVファイルの読み込み
import_data = pd.read_csv("dump_data_mine.csv")
import_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-04-13 00:54:07,201904,商品X,2400.0,長谷川舞,サンプルかな,X市,sample@example.com,2021-10-31
1,2019-05-01 14:22:42,201905,商品P,1600.0,山田千代,サンプルかな,X市,sample@example.com,2018-04-30
2,2019-05-02 09:04:01,201905,商品E,500.0,鈴木真綾,サンプルかな,X市,sample@example.com,2021-01-31
3,2019-04-10 03:01:42,201904,商品T,2000.0,伊藤康弘,サンプルかな,X市,sample@example.com,2019-04-30
4,2019-05-09 07:55:30,201905,商品E,500.0,渡辺陽一,サンプルかな,X市,sample@example.com,2022-02-28
...,...,...,...,...,...,...,...,...,...
801,2019-05-15 08:45:12,201905,商品E,500.0,遠藤洋介,サンプルかな,X市,sample@example.com,2018-02-05
802,2019-04-27 19:28:07,201904,商品J,1000.0,清水涼平,サンプルかな,X市,sample@example.com,2020-10-31
803,2019-01-22 11:39:36,201901,商品K,1100.0,伊藤康弘,サンプルかな,X市,sample@example.com,2019-04-30
804,2019-01-17 20:03:45,201901,商品T,2000.0,山本聡太郎,サンプルかな,X市,sample@example.com,2018-08-31


In [35]:
# pivot_tableを使い購入月、商品ごとの購入回数を集計
byItem = import_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)
byItem

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,2,2,4,7,5,2,6,6,8,...,1,5,7,4,5,4,3,5,4,2
201902,3,5,1,7,7,3,1,3,4,5,...,5,4,7,3,4,5,4,4,4,6
201903,2,5,2,3,7,9,3,8,5,2,...,3,4,4,4,3,3,5,4,6,7
201904,4,8,8,5,5,2,8,3,1,4,...,6,3,2,7,5,4,6,7,1,4
201905,5,1,2,4,10,6,9,2,3,6,...,2,6,6,5,7,6,5,7,7,6
201906,4,10,4,2,1,6,5,5,5,9,...,3,4,4,2,4,8,6,5,10,3
201907,3,3,4,1,3,7,3,2,3,7,...,6,3,4,6,2,2,4,0,6,8


In [30]:
# 月ごとの商品別売上金額の集計
byPrice = import_data.pivot_table(index="purchase_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,300.0,400.0,600.0,1600.0,3500.0,3000.0,1400.0,4800.0,5400.0,8000.0,...,1700.0,9000.0,13300.0,8000.0,10500.0,8800.0,6900.0,12000.0,10000.0,5200.0
201902,300.0,1000.0,300.0,2800.0,3500.0,1800.0,700.0,2400.0,3600.0,5000.0,...,8500.0,7200.0,13300.0,6000.0,8400.0,11000.0,9200.0,9600.0,10000.0,15600.0
201903,200.0,1000.0,600.0,1200.0,3500.0,5400.0,2100.0,6400.0,4500.0,2000.0,...,5100.0,7200.0,7600.0,8000.0,6300.0,6600.0,11500.0,9600.0,15000.0,18200.0
201904,400.0,1600.0,2400.0,2000.0,2500.0,1200.0,5600.0,2400.0,900.0,4000.0,...,10200.0,5400.0,3800.0,14000.0,10500.0,8800.0,13800.0,16800.0,2500.0,10400.0
201905,500.0,200.0,600.0,1600.0,5000.0,3600.0,6300.0,1600.0,2700.0,6000.0,...,3400.0,10800.0,11400.0,10000.0,14700.0,13200.0,11500.0,16800.0,17500.0,15600.0
201906,400.0,2000.0,1200.0,800.0,500.0,3600.0,3500.0,4000.0,4500.0,9000.0,...,5100.0,7200.0,7600.0,4000.0,8400.0,17600.0,13800.0,12000.0,25000.0,7800.0
201907,300.0,600.0,1200.0,400.0,1500.0,4200.0,2100.0,1600.0,2700.0,7000.0,...,10200.0,5400.0,7600.0,12000.0,4200.0,4400.0,9200.0,0.0,15000.0,20800.0


In [36]:
# 月ごとの顧客別購入回数の集計
byCustomer = import_data.pivot_table(index="purchase_month", columns="顧客名", aggfunc="size", fill_value=0)
byCustomer

顧客名,三浦直樹,中島加奈,中村治,中村裕樹,井上真綾,伊藤康弘,伊藤零,佐々木京助,佐々木千代,佐々木幹,...,藤原裕樹,遠藤洋介,遠藤零,鈴木涼平,鈴木直樹,鈴木真綾,鈴木裕太,長谷川舞,阿部学,高橋結衣
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,2,2,1,1,7,3,1,1,1,3,...,4,1,3,2,0,3,3,1,1,2
201902,1,2,0,4,1,4,2,3,4,0,...,4,3,4,6,3,3,1,3,2,0
201903,3,4,1,4,2,3,4,5,0,2,...,2,2,3,6,2,3,2,2,4,2
201904,5,4,3,1,1,6,2,1,5,2,...,6,1,1,3,1,2,1,7,1,1
201905,4,4,5,2,2,3,0,2,2,0,...,4,3,5,7,3,6,2,0,5,4
201906,4,2,4,2,4,3,2,3,2,2,...,2,1,4,3,5,2,2,3,2,3
201907,3,0,0,3,1,3,3,2,1,1,...,1,1,3,6,3,3,1,1,4,1


In [41]:
# 月ごとの地域別購入回数の集計
byRegion = import_data.pivot_table(index="purchase_month", columns="地域", aggfunc="size", fill_value=0)
byRegion

地域,X市
purchase_month,
201901,114
201902,115
201903,112
201904,111
201905,129
201906,125
201907,100


In [38]:
import_data['地域'].unique()

array(['X市'], dtype=object)

In [43]:
# 地域データの(ユニークな値)を取得
away_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name", right_on="顧客名", how="right")
away_data[away_data["purchase_date"].isnull()][["顧客名", "メールアドレス", "登録日"]]

,顧客名,メールアドレス,登録日


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')